In [1]:
!pip install langchain faiss-cpu openai tiktoken pypdf

In [2]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

docs_path = './input'
all_docs = []

for file in os.listdir(docs_path):
    if file.endswith('.pdf'):
        loader = PyPDFLoader(os.path.join(docs_path, file))
        pages = loader.load()
        splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
        docs = splitter.split_documents(pages)
        all_docs.extend(docs)

print(f"Loading completed, total {len(all_docs)} segments")

/Users/xujing/miniconda3/envs/py3/lib/python3.10/site-packages/pypdf/_crypt_providers/_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4


加载完成，共 418 段


In [3]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(all_docs, embedding_model)
vectorstore.save_local("faiss_index")

/var/folders/nn/vxf45hss6vv4b_t4xkxhkm240000gn/T/ipykernel_47170/960842804.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/Users/xujing/miniconda3/envs/py3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from langchain_community.llms import Ollama
from langchain.retrievers.multi_query import MultiQueryRetriever

llm = Ollama(model="deepseek-r1:8b")

# Multi-perspective question generation + multi-round retrieval
multi_retriever = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(search_kwargs={"k": 4}),
    llm=llm
)

/var/folders/nn/vxf45hss6vv4b_t4xkxhkm240000gn/T/ipykernel_47170/664163176.py:4: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="deepseek-r1:8b")


In [5]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=multi_retriever,
    return_source_documents=True
)

query = "What deep learning methods are mentioned in the papers?"
result = qa_chain(query)

print("🧠 Answer:")
print(result["result"])
print("\n📚 Sources:")
for doc in result["source_documents"]:
    print(f"- Filename: {doc.metadata.get('source', 'Unknown')}  | Page: {doc.metadata.get('page', 'Unknown')}")

/var/folders/nn/vxf45hss6vv4b_t4xkxhkm240000gn/T/ipykernel_47170/4291094833.py:10: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain(query)


🧠 回答：
<think>
好的，我现在需要回答用户的问题：“论文中提到了哪些深度学习方法？”首先，我得仔细看一下提供的上下文内容。

论文中提到的模型部分主要是CSP（Center-Part Suppression），这是他们提出的方法。接着看到对比的是Fr-RCNN*、MS-CNN和他们的基线模型。这些应该都是相关的检测或识别方法。另外，在技术细节里提到了R-FoCN，可能是指R-FCN的一种变体或者相关工作，比如使用多尺度特征融合的结构。

在训练部分提到的深度学习组件包括ResNet（他们用的是ResNet-50），以及各种归一化层如BN、IN和LN。还有激活函数ReLU，损失函数是Smooth L1 Loss，并调整了超参数r值。提出的ACSP方法结合了BN、IN和LN，各占一定比例。

然后在相关的研究工作中引用了很多论文的方法，比如RCNN系列（Faster RCNN, R-FCN）、SSD、YOLO等传统CNN检测器，还有Cityscapes数据集处理中的多尺度特征融合策略。这里需要注意用户的问题可能不仅限于他们自己的方法，而是整个文献中提到的所有深度学习技术。

总结一下，主要的方法包括CSP及其变体（如ACSP），对比的模型有Fr-RCNN、MS-CNN和基线Faster R-CNN，并提到了R-FoCN作为他们的改进基础。归一化层方面涉及Batch Normalization, Instance Normalization, Layer Normalization等。多尺度特征融合参考了CSP的工作，但可能在其他部分也有应用。此外，在讨论相关工作时提到的还有各种经典的CNN检测方法如Faster R-CNN、R-FCN、SSD、YOLO，以及语义分割和边缘检测相关的模型。

需要确保没有遗漏任何明显的方法名称，并且正确解释每个术语的关系。例如，“conv4 3”可能属于CSP或他们的基线部分，而提出的ACSP方法应该包括多层卷积等操作来融合不同归一化层的结果。
</think>
根据提供的论文摘要和上下文信息：

1. **Fr-RCNN** (Region-based Faster R-CNN)：用于目标检测的基准模型之一，具有两阶段处理流程（RPN 和 FRCNN）。

2. **MS-CNN** (Multi-Scale CNN)：另一个

In [6]:
query = "Please summarize the main contributions and innovations of the CCHI paper?"
result = qa_chain(query)

print("🧠 Answer:")
print(result["result"])
print("\n📚 Sources:")
for doc in result["source_documents"]:
    print(f"- Filename: {doc.metadata.get('source', 'Unknown')}  | Page: {doc.metadata.get('page', 'Unknown')}")

🧠 回答：
<think>
嗯，用户让我总结一篇名为“CSP”的论文的main contributions部分。首先需要仔细阅读提供的上下文材料，找出关键信息。

看到了很多关于行人检测的数据表、实验设置和讨论段落，里面提到了一些技术细节比如分辨率调整、不同归一化方法的影响等，但没有直接提到标题或具体论文名称。不过用户的问题中提到了“CCHI”这篇论文，我需要确认上下文是否与之对应。

从内容来看，表格显示了行人检测的mAP和FPS数据比较，特别是他们提出的方法在不同分辨率下表现良好，尤其是在640×1280时达到最佳性能。还讨论了使用Switchable Normalization (SN)提升收敛性和稳定性，以及提到不需要预定义先验框就能处理高纵横比的目标。

这些点确实体现了该方法的创新性：通过调整输入分辨率解决了关键问题；引入自适应归一化策略增强了网络对不同特征尺度的适应能力；不依赖于anchor-based机制提高了泛化性。实验结果也显示了在CityPersons和COCO数据集上的优势，特别是在高纵横比目标检测方面。

需要确认这些信息是否准确对应到用户提到的“CCHI”论文的核心贡献上，并检查是否有其他相关点被遗漏或强调。此外要注意表格中的比较对象如Fr-RCNN、MS-CNN和他们的基线方法，以及对现有方法（如TLL）的对比分析。
</think>
根据提供的上下文内容，可以总结出 CCHI 论文的主要贡献和创新点如下：

1. **输入分辨率的重要性**：作者通过实验发现，在行人检测中使用特定的输入分辨率（例如 640×1280）能够显著提升模型性能。他们的方法在该分辨率下表现最佳，并且比原始分辨率有更好的收敛性。
   
2. **自适应归一化策略 (Switchable Normalization)**：他们提出了一种新的归一化方法，称为 Switchable Normalization (SN)，通过动态调整不同层使用哪种归一化方式（包括 BatchNorm、InstanceNorm 和 LayerNorm）来提升模型的性能。实验表明 SN 比单一的 BN 或 LN 方法更有效。

3. **无需先验框**：他们提出的方法不需要像传统 anchor-based 方法那样依赖预定义的边界框，这使得其具有更强的泛化能力，并且在高纵横比目标检测方面

In [ ]:
llm = Ollama(model="llama3.2:latest")
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=multi_retriever,
    return_source_documents=True
)

query = "What deep learning methods are mentioned in the papers?"
result = qa_chain(query)

print("🧠 Answer:")
print(result["result"])
print("\n📚 Sources:")
for doc in result["source_documents"]:
    print(f"- Filename: {doc.metadata.get('source', 'Unknown')}  | Page: {doc.metadata.get('page', 'Unknown')}")

🧠 回答：
The paper mentions several deep learning methods:

1. R-FCN (Region-based Fully Convolutional Networks)
2. YOLO V3
3. CSP (Class-Specific Pixel Loss) for pedestrian detection
4. SN (Scale Normalization) for normalization
5. BN (Batch Normalization) for normalization
6. LN (Layer Normalization) for normalization

Note that some of these methods are mentioned as being compared or contrasted with others, rather than being the primary focus of the paper.

📚 来源：
- 文件名: ./input/Mao_What_Can_Help_CVPR_2017_paper.pdf  | 页码: 4
- 文件名: ./input/2019 cvpr CSP.pdf  | 页码: 17
- 文件名: ./input/Mao_What_Can_Help_CVPR_2017_paper.pdf  | 页码: 0
- 文件名: ./input/Mao_What_Can_Help_CVPR_2017_paper.pdf  | 页码: 8
- 文件名: ./input/CCHI.pdf  | 页码: 4
- 文件名: ./input/CCHI.pdf  | 页码: 4
- 文件名: ./input/CCHI.pdf  | 页码: 3
- 文件名: ./input/2019 cvpr CSP.pdf  | 页码: 0
- 文件名: ./input/ACSP.pdf  | 页码: 4
- 文件名: ./input/CCHI.pdf  | 页码: 3
- 文件名: ./input/ACSP.pdf  | 页码: 4
- 文件名: ./input/Mao_What_Can_Help_CVPR_2017_paper.pdf  | 页码: 9
-

: 